In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import os 
import sys 
import warnings 
import random
from pprint import pprint as pp
from dotenv import load_dotenv
import os
from huggingface_hub import whoami, HfFolder

import gc

import torch
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from torch.utils.data.dataloader import DataLoader

from transformers import BitsAndBytesConfig
from transformers import DataCollatorForLanguageModeling
from transformers import AutoTokenizer, AutoModelForCausalLM 
from transformers import set_seed, Seq2SeqTrainer, LlamaTokenizer

from datasets import Dataset, DatasetDict

from peft import LoraConfig, get_peft_model


c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "Models/Llama-3.2-1B-Instruct/Model"
tokenizer_id = "Models/Llama-3.2-1B-Instruct/Tokenizer"

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)

# model.save_pretrained("Models/Llama-3.2-1B-Instruct/Model")
# tokenizer.save_pretrained("Models/Llama-3.2-1B-Instruct/Tokenizer")

In [3]:
warnings.filterwarnings("ignore") 
# log_hf()
load_dotenv("env_vars.env")

set_seed(42)
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)


In [4]:
batch_size = 8
n_epochs = 2
lr = 1e-5
lora_r = 8

In [5]:
########################################################## DATA WORK
print("_________________________________")
print("Preapring the Data")


df = pd.read_csv("df_from_exp_to_imp.csv")

base_prompt = """You are a social media content moderator.
INSTRUCTION: The following is a social media message that needs to be classified with the label HATEFUL or NOT HATEFUL.
MESSAGE: {}
OUTPUT AND FORMAT: your output should be just the label."""

_________________________________
Preapring the Data


In [6]:
tokenizer.SPECIAL_TOKENS_ATTRIBUTES

['bos_token',
 'eos_token',
 'unk_token',
 'sep_token',
 'pad_token',
 'cls_token',
 'mask_token',
 'additional_special_tokens']

In [12]:
# tokenizer.encode(tokenizer.pad_token)

In [13]:
print(tokenizer.mask_token)

None


In [14]:
print(tokenizer.pad_token)
if tokenizer.pad_token is None: tokenizer.pad_token = '<|finetune_right_pad_id|>'

None


In [20]:
print(tokenizer.pad_token)
print(type(tokenizer.encode(tokenizer.pad_token, add_special_tokens=False))[0])

<|finetune_right_pad_id|>
list[0]


In [22]:
tokenizer.encode(tokenizer.pad_token, add_special_tokens=True)[0]

128000

In [21]:
tokenizer.encode(tokenizer.pad_token, add_special_tokens=False)[0]

128004

In [17]:
tokens = tokenizer.encode("Hello, how are you?", return_tensors="pt")
print(tokens.shape)
second = tokens.fill_(-100)
tok = tokenizer.encode("Hello, how are you?", return_tensors="pt")

# torch_tensor = torch.tensor(tokens)


torch.Size([1, 7])


In [267]:
both = torch.cat((tokens,tok), dim=1)
both

tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100, 128000,   9906,
             11,   1268,    527,    499,     30]])

In [245]:
second

tensor([[-100, -100, -100, -100, -100, -100, -100]])

In [269]:
# out = model(both)
# print(out)

In [223]:
tokenizer.pad_token

'<|end_of_text|>'

In [ ]:
def translate_class_to_label(class_):

    translation_dict = {"not_hate": "NOT HATEFUL",
                        "explicit_hate": "HATEFUL",
                        "implicit_hate": "HATEFUL"}

    translated_label = translation_dict[class_]

    return translated_label


In [221]:
   

#### Attaching the prompt to the clean post

df["formatted_prompt"] = df["clean_post"].apply(format_prompt)
df["label"] = df["class"].apply(translate_class_to_label)

# ### Turning the Df into a DatasetDict

t_1 = []
t_2 = []

for split in df["split"].unique():

    split_df_1 = df[(df["split"] == split) & (df["time"] == 1)]
    split_df_2 = df[(df["split"] == split) & (df["time"] == 2)]

    hf_split_1 = Dataset.from_pandas(split_df_1)
    hf_split_2 = Dataset.from_pandas(split_df_2)
    
    t_1.append(hf_split_1)
    t_2.append(hf_split_2)

hf_time_1 = DatasetDict({t_1[0]["split"][0]: t_1[0], 
                        t_1[1]["split"][0]: t_1[1],
                        t_1[2]["split"][0]: t_1[2]})

hf_time_2 = DatasetDict({t_2[0]["split"][0]: t_2[0], 
                        t_2[1]["split"][0]: t_2[1],
                        t_2[2]["split"][0]: t_2[2]})




In [11]:
########################################################## TOKENIZER WORK

hf_time_1 = hf_time_1.map(preprocess_and_tokenize, input_columns=["formatted_prompt", "label"], batched=False)
hf_time_2 = hf_time_2.map(preprocess_and_tokenize, input_columns=["formatted_prompt", "label"], batched=False)



Map: 100%|██████████| 1020/1020 [00:00<00:00, 1156.90 examples/s]


In [27]:
hf_time_1["train"][0]["label"]

'NOT HATEFUL'

In [28]:
tokenizer.eos_token

'<|endoftext|>'

In [30]:
tokenizer.pad_token_id

151643

In [33]:
tokenizer.decode(151643)
end_of_text_token = 151643

In [41]:
hate_encoding = tokenizer.encode("HATEFUL")


In [42]:
not_hate_encoding = tokenizer.encode("NOT HATEFUL")


In [59]:
example = hf_time_1["train"][0]
example.keys()

dict_keys(['clean_post', 'post', 'class', 'implicit_class', 'extra_implicit_class', 'target', 'implied_statement', 'split', 'time', 'formatted_prompt', 'label', '__index_level_0__', 'input_ids', 'attention_mask'])

In [61]:
end_prompt = example["input_ids"][0].index(end_of_text_token)
end_prompt

90

In [71]:
label_encoding = tokenizer.encode(example["label"] + tokenizer.eos_token)
label_encoding

[14065, 472, 2336, 49636, 151643]

In [84]:
tokenizer.decode(14065)

'NOT'

In [87]:
start_answer = end_prompt-(len(label_encoding)+1)

In [ ]:
tokenizer.decode(example["input_ids"][0][: start_answer])

In [88]:
tokenizer.decode(example["input_ids"][0][start_answer : end_prompt - 1])

'NOT HATEFUL<|im_end|>'

In [138]:
example.keys()

dict_keys(['clean_post', 'post', 'class', 'implicit_class', 'extra_implicit_class', 'target', 'implied_statement', 'split', 'time', 'formatted_prompt', 'label', '__index_level_0__', 'input_ids', 'attention_mask'])

In [ ]:
# base_prompt



In [190]:
tokenizer.decode(198)

'\n'

In [ ]:
if tokenizer.pad_token is None: tokenizer.pad_token = '<|finetune_right_pad_id|>'

def preprocess_and_tokenize(clean_post, label, base_prompt=base_prompt, max_length=312):

    # if type(label) != list:
    #     label = [label]
    # if type(clean_post) != list:
    #     clean_post = [clean_post]
    
    prompt_plus_messages = base_prompt.format(clean_post)
    # pp(prompt_plus_messages)
    # pp(label)
    messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": prompt_plus_messages},
            {"role": "assistant", "content": label.strip("\n")}
        ]

    # print(messages)
    chat_template = tokenizer.apply_chat_template(messages, tokenize=False, continue_final_message=False, add_special_tokens=False).rstrip()
    # print(chat_template)

    # why is the chat template putting a new line at the end of the end of sequence
    # pp(chat_template)
    input_ids_tokenized = tokenizer(chat_template, return_tensors="pt", add_special_tokens=False, padding="max_length", max_length=max_length)["input_ids"]

    # getting the normal text just to know how much we need to add to the left as -100 and right as pad token
    input_ids_shape = tokenizer(chat_template, return_tensors="pt", add_special_tokens=False, padding=False)["input_ids"]
    # print(input_ids_tokenized)

    # getting the label target to only predict the actual label and ignore the prompt
    labels_tokenized = tokenizer(label + tokenizer.eos_token, add_special_tokens=True, return_tensors="pt")["input_ids"]
    shape = input_ids_shape.shape[1] - labels_tokenized.shape[1]
    zeros = torch.zeros((1, shape), dtype=labels_tokenized.dtype, device=labels_tokenized.device)
    zeros.fill_(-100) # acc to llama docs
    labels_left_padded = torch.cat([zeros, labels_tokenized], dim=1)

    eos_n = input_ids_tokenized.shape[1] - labels_left_padded.shape[1]
    eos_n_tensor = torch.zeros((1, eos_n), dtype=labels_tokenized.dtype, device=labels_tokenized.device)
    eos_n_tensor.fill_(tokenizer.eos_token_id)
    labels_padded = torch.cat([labels_left_padded, eos_n_tensor], dim=1)

    # print(labels_padded.shape == input_ids_tokenized.shape)

    # shifting because we dont predict the first token
    input_ids_tokenized_left_shifted = input_ids_tokenized[:, :-1]
    labels_tokenized_right_shifted = labels_padded[:, 1:]

    attention_mask = input_ids_tokenized_left_shifted != tokenizer.pad_token_id
    
    return {
        "input_ids": input_ids_tokenized_left_shifted,
        "labels": labels_tokenized_right_shifted,
        "attention_mask": attention_mask
    }

In [214]:
hf_time_1 = hf_time_1.map(preprocess_and_tokenize, input_columns=["clean_post", "label"], batched=False)


Map: 100%|██████████| 720/720 [00:01<00:00, 622.72 examples/s]


In [ ]:
hf_time_1.set_format("torch")
hf_time_2.set_format("torch")

cols_to_remove = ["clean_post", "post", "class", "implicit_class", "extra_implicit_class", "target", "implied_statement", "split", "time", "formatted_prompt", "label", "__index_level_0__"]

for split in hf_time_1:
    if split != "test":
        hf_time_1[split] = hf_time_1[split].remove_columns(cols_to_remove)
        hf_time_2[split] = hf_time_2[split].remove_columns(cols_to_remove)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


hf_time_1_train_loader = DataLoader(hf_time_1["train"], collate_fn=data_collator, batch_size=batch_size)
hf_time_1_validation_loader = DataLoader(hf_time_1["validation"], collate_fn=data_collator, batch_size=batch_size)
hf_time_1_test_loader = DataLoader(hf_time_1["test"], collate_fn=data_collator, batch_size=batch_size)

hf_time_2_train_loader = DataLoader(hf_time_2["train"], collate_fn=data_collator, batch_size=batch_size)
hf_time_2_validation_loader = DataLoader(hf_time_2["validation"], collate_fn=data_collator, batch_size=batch_size)
hf_time_2_test_loader = DataLoader(hf_time_2["test"], collate_fn=data_collator, batch_size=batch_size)

# ### So far, created the prompt, did the messages with the prompt and answer in place. Applied to chat template and tokenized 


In [8]:
########################3#################### MODEL WORK

print("_________________________________")
print("Loading the model and model config")

bnb_config = BitsAndBytesConfig(  
                                load_in_4bit= True,
                                bnb_4bit_quant_type= "nf4",
                                bnb_4bit_compute_dtype= torch.bfloat16,
                                bnb_4bit_use_double_quant= True,
                            )

model = AutoModelForCausalLM.from_pretrained(model_id,
                                            torch_dtype=torch.bfloat16,
                                            device_map="auto",
                                            quantization_config=bnb_config
                                            )

# to deal with the fact that we dont make the first token prediction??


model_size_before = sum(t.numel() for t in model.parameters())
print("Model Size before LoRA", model_size_before)
print(model)
print()

lora_alpha = lora_r*2
config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
    lora_dropout=0.1,
    bias="none",
)

model = get_peft_model(model, config)
print("Model After LoRA")
model.print_trainable_parameters()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model.to(device)


loss_fn = CrossEntropyLoss()
optimizer = AdamW((param for param in model.parameters() if param.requires_grad), lr=lr)


_________________________________
Loading the model and model config
Model Size before LoRA 315119488
Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layer

In [ ]:
print("_________________________________")
print("Training the model")
print()

for epoch in range(n_epochs):

    torch.cuda.empty_cache()
    gc.collect()
    model.train()

    print("Epoch: ", epoch)
    losses = []

    for i, batch in enumerate(hf_time_1_train_loader):
        if i > 0:
            continue

        torch.cuda.empty_cache()
        gc.collect()

        print("\tBatch: ", i)
        # print(batch)
        batch.to(device)
        # print(batch.keys())
        # print(batch["input_ids"].shape)
        # print(batch["attention_mask"].shape)
        # print(batch["labels"].shape)


        batch = {k:torch.squeeze(v) for k,v in batch.items()}

        # print(batch["input_ids"].shape)
        # print(batch["attention_mask"].shape)
        # print(batch["labels"].shape)


        output = model(**batch)
        logits = output.logits
        loss = loss_fn(logits, batch["labels"])

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        losses.append(loss.detach().item())

        print(batch.keys())
        print(loss.detach().item())
        print(output.logits.shape)
        print(output.probas)

        if i > 3:
            continue

    epoch_loss = sum(losses)/len(hf_time_1_train_loader)
    print(f"Epoch {epoch} Loss: {epoch_loss}")

    model.eval()
    with torch.no_grad():  

        torch.cuda.empty_cache()
        gc.collect()

        val_losses = []

        for i, batch in enumerate(hf_time_1_validation_loader):
            if i > 0:
                continue
            batch.to(device)
            batch = {k:torch.squeeze(v) for k,v in batch.items()}

            output = model(**batch)
            logits = output.logits
            val_loss = loss_fn(logits, batch["labels"])

            val_losses.append(val_loss.detach().item())

        val_loss_epoch = sum(val_losses)/len(hf_time_1_validation_loader)
        print(f"Epoch {epoch} Validation Loss: {val_loss_epoch}")
print()


In [ ]:
print("_________________________________")
print("Testing the model")
for i, test_batch in enumerate(hf_time_1["test"]):

    if i > 0:
        break
    
    text = test_batch["formatted_prompt"]
    tokenized_chat_template, messages_list = preprocess_and_tokenize(text, label=False, add_generation_prompt=True, output_messages_list=True)
    output = model.generate(**tokenized_chat_template.to(device))
    pred = tokenizer.decode(output[0], skip_special_tokens=True)
    
    print(text)
    print(tokenized_chat_template)
    print(output)
    print(pred)

print("CHECKING GENERATION")
print(messages_list)

print(tokenized_chat_template)
print(output)

print(type(output))
print(output.shape)

print("_________________________________")
print("Saving the model and Tokenizer")
model_name = model_id.split("/")[-1]
model.save_pretrained(f"alberto-lorente/{model_name}_test")
tokenizer.save_pretrained(f"alberto-lorente/{model_name}_test")

print("RUN SUCCESSFULLY")
